## Transfer Learning using MNIST data
To illustrate the power and concept of transfer learning, we will train a CNN on just the digits 5,6,7,8,9.  Then we will train just the last layer(s) of the network on the digits 0,1,2,3,4 and see how well the features learned on 5-9 help with classifying 0-4.

Adapted from https://github.com/fchollet/keras/blob/master/examples/mnist_transfer_cnn.py

In [1]:
import os, random, numpy as np, tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'

def set_seed(seed_value=1234):
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
set_seed()
    

In [2]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [3]:
#used to help some of the timing functions
now = datetime.datetime.now

In [4]:
# set some parameters
batch_size = 128
num_classes = 5
epochs = 10

In [5]:
# set some more parameters
img_rows, img_cols = 28, 28

In [6]:
## This just handles some variability in how the input data is loaded

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [7]:
## To simplify things, write a function to include all the training steps
## As input, function takes a model, training set, test set, and the number of classes
## Inside the model object will be the state about which layers we are freezing and which we are training

def train_model(model, train, test, num_classes, epochs=5):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='sgd',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [8]:
# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets: one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

11490434/11490434 [==============================] - 2s 0us/step


In [9]:
# Define the "feature" layers.  These are the early layers that we expect will "transfer"
# to a new problem.  We will freeze these layers during the fine-tuning process

feature_layers = [
    Conv2D(16, kernel_size=(3, 3),
           padding='valid',
           input_shape=input_shape, activation='relu'),
    Conv2D(16, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten()
]

In [10]:
# Define the "classification" layers.  These are the later layers that predict the specific classes from the features
# learned by the feature layers.  This is the part of the model that needs to be re-trained for a new problem

classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]

In [11]:
# We create our model by combining the two sets of layers as follows
model = Sequential(feature_layers + classification_layers)

In [12]:
# Let's take a look
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 16)        0         
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               295040    
                                                        

In [13]:
# Now, let's train our model on the digits 5,6,7,8,9

train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes, epochs=epochs)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/10
230/230 [==============================] - 3s 9ms/step - loss: 0.9881 - accuracy: 0.6249 - val_loss: 0.3103 - val_accuracy: 0.9072
Epoch 2/10
230/230 [==============================] - 1s 6ms/step - loss: 0.4033 - accuracy: 0.8608 - val_loss: 0.2155 - val_accuracy: 0.9319
Epoch 3/10
230/230 [==============================] - 1s 6ms/step - loss: 0.3152 - accuracy: 0.8972 - val_loss: 0.1770 - val_accuracy: 0.9461
Epoch 4/10
230/230 [==============================] - 1s 6ms/step - loss: 0.2718 - accuracy: 0.9101 - val_loss: 0.1534 - val_accuracy: 0.9512
Epoch 5/10
230/230 [==============================] - 1s 6ms/step - loss: 0.2446 - accuracy: 0.9190 - val_loss: 0.1387 - val_accuracy: 0.9576
Epoch 6/10
230/230 [==============================] - 1s 6ms/step - loss: 0.2190 - accuracy: 0.9288 - val_loss: 0.1257 - val_accuracy: 0.9595
Epoch 7/10
230/230 [==============================] - 1s 6ms/step - loss: 0.

### Freezing Layers
Keras allows layers to be "frozen" during the training process.  That is, some layers would have their weights updated during the training process, while others would not.  This is a core part of transfer learning, the ability to train just the last one or several layers.

Note also, that a lot of the training time is spent "back-propagating" the gradients back to the first layer.  Therefore, if we only need to compute the gradients back a small number of layers, the training time is much quicker per iteration.  This is in addition to the savings gained by being able to train on a smaller data set.

### Looking Inside Model

## [Code for Viewing Layers](https://docs.google.com/document/d/1iACjBPWTWvXTSecBNHx1xEtODcHSpnU6HFY82b4b6Ys)

#### Copy and Paste the code here!

### Freeze CNN Layers (Low-level or Early Layers)

In [14]:
# Freeze only the 
for l in feature_layers:
    l.trainable = False

Observe below the differences between the number of *total params*, *trainable params*, and *non-trainable params*.

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 16)        2320      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 16)        0         
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dense (Dense)               (None, 128)               295040    
                                                        

In [16]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes, epochs=epochs//4)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/2
240/240 [==============================] - 2s 7ms/step - loss: 0.4832 - accuracy: 0.8475 - val_loss: 0.1246 - val_accuracy: 0.9658
Epoch 2/2
240/240 [==============================] - 1s 5ms/step - loss: 0.2347 - accuracy: 0.9297 - val_loss: 0.0916 - val_accuracy: 0.9745
Training time: 0:00:03.643100
Test score: 0.09162917733192444
Test accuracy: 0.9745086431503296


Note that after a single epoch, we are already achieving results on classifying 0-4 that are comparable to those achieved on 5-9 after 5 full epochs.  This despite the fact the we are only "fine-tuning" the last layer of the network, and all the early layers have never seen what the digits 0-4 look like.

Also, note that even though nearly all (590K/600K) of the *parameters* were trainable, the training time per epoch was still much reduced.  This is because the unfrozen part of the network was very shallow, making backpropagation faster. 

## Exercise
### To do:
- Now write code to reverse this training process.  That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [17]:
# Reinitialize the model
model = Sequential(feature_layers + classification_layers)

# Train on the digits 0-4
train_model(model,
      (x_train_lt5, y_train_lt5),
      (x_test_lt5, y_test_lt5), num_classes, epochs=epochs)

# Freeze the feature layers
for l in feature_layers:
  l.trainable = False

# Train on the digits 5-9
train_model(model,
      (x_train_gte5, y_train_gte5),
      (x_test_gte5, y_test_gte5), num_classes, epochs=epochs//4)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/10
240/240 [==============================] - 2s 7ms/step - loss: 0.1887 - accuracy: 0.9420 - val_loss: 0.0787 - val_accuracy: 0.9774
Epoch 2/10
240/240 [==============================] - 1s 6ms/step - loss: 0.1658 - accuracy: 0.9512 - val_loss: 0.0711 - val_accuracy: 0.9786
Epoch 3/10
240/240 [==============================] - 1s 6ms/step - loss: 0.1582 - accuracy: 0.9518 - val_loss: 0.0670 - val_accuracy: 0.9807
Epoch 4/10
240/240 [==============================] - 1s 6ms/step - loss: 0.1476 - accuracy: 0.9564 - val_loss: 0.0611 - val_accuracy: 0.9829
Epoch 5/10
240/240 [==============================] - 1s 6ms/step - loss: 0.1440 - accuracy: 0.9559 - val_loss: 0.0623 - val_accuracy: 0.9829
Epoch 6/10
240/240 [==============================] - 1s 6ms/step - loss: 0.1309 - accuracy: 0.9600 - val_loss: 0.0546 - val_accuracy: 0.9842
Epoch 7/10
240/240 [==============================] - 1s 6ms/step - loss: 0.